# MODEL TRAINING - SET 2

### 1.1 Import Data and Required Packages
#### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [219]:
#Importing recquired packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

##### Importing Modelling Libraries

In [220]:
#Importing ML Packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.svm import SVR                          
from sklearn.tree import DecisionTreeRegressor 
from sklearn.neighbors import KNeighborsRegressor   

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.multioutput import MultiOutputRegressor

from xgboost import XGBRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.optimizers import Adam
import datetime

#### Importing CSV data as Pandas dataframe

In [221]:
#Importing CSV data as Pandas dataframe
df=pd.read_csv('dataset/Extracted_CrudeData.csv')

#Show top 5 rows of the dataframe
df.head()

,Crude Name,StdLiquidDensity (kg/m3),SulfurByWt (%),ConradsonCarbonByWt (%),NitrogenByWt (%),Distillation Mass @ X Pct (C)@ 1 (%) - TBP,Distillation Mass @ X Pct (C)@ 5 (%) - TBP,Distillation Mass @ X Pct (C)@ 10 (%) - TBP,Distillation Mass @ X Pct (C)@ 30 (%) - TBP,Distillation Mass @ X Pct (C)@ 50 (%) - TBP,...,AromByWt (%),NaphthenesByWt (%),ParaffinsByWt (%),KinematicViscosity (cSt)@ 37.78 (C),KinematicViscosity (cSt)@ 98.89 (C),KinematicViscosity (cSt)@ 20 (C),KinematicViscosity (cSt)@ 40 (C),KinematicViscosity (cSt)@ 50 (C),KinematicViscosity (cSt)@ 100 (C),KinematicViscosity (cSt)@ 150 (C)
0,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,24.571141,41.011754,34.417105,1.7573028543796,0.833438,2.091216,1.700584,1.443025,0.824673,NaN
1,AlJurf-2014,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,...,42.157942,36.772998,21.069060,7.05914911333451,2.025492,11.190540,6.703264,5.225721,1.991830,NaN
2,AlJurf-2015,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,...,39.122494,39.730427,21.147079,6.77137102006056,2.010330,10.544402,6.437194,5.049799,1.977528,NaN
3,Ashtart-2015,870.860260,1.002298,5.194113,0.233022,17.461417,84.608073,121.721566,248.007854,358.275744,...,40.484153,44.017708,15.498139,6.6515298282988,1.990580,10.208774,6.317605,4.927973,1.959541,NaN
4,Azeri-Light-Supsa-2019,846.458826,0.147956,0.735866,0.113251,4.265112,81.831627,120.955400,229.274959,320.215604,...,21.364167,58.922758,19.713075,3.93702798465944,1.841686,5.117420,3.810679,3.255558,1.818677,NaN


In [222]:
#Shape of dataset
df.shape

(53, 24)

In [223]:
#Renaming columns for better readability
df.rename(columns={
    'Crude Name': 'Crude_Name',
    'StdLiquidDensity (kg/m3)': 'StdLiqDensity_kgm3',
    'SulfurByWt (%)': 'SulfurWt_pct',
    'ConradsonCarbonByWt (%)': 'ConradsonCarbon_wt_pct',
    'NitrogenByWt (%)': 'NitrogenWt_pct',
    
    'Distillation Mass @ X Pct (C)@ 1 (%) - TBP': 'TBP_TempAt_1pct',
    'Distillation Mass @ X Pct (C)@ 5 (%) - TBP': 'TBP_TempAt_5pct',
    'Distillation Mass @ X Pct (C)@ 10 (%) - TBP': 'TBP_TempAt_10pct',
    'Distillation Mass @ X Pct (C)@ 30 (%) - TBP': 'TBP_TempAt_30pct',
    'Distillation Mass @ X Pct (C)@ 50 (%) - TBP': 'TBP_TempAt_50pct',
    'Distillation Mass @ X Pct (C)@ 70 (%) - TBP': 'TBP_TempAt_70pct',
    'Distillation Mass @ X Pct (C)@ 90 (%) - TBP': 'TBP_TempAt_90pct',
    'Distillation Mass @ X Pct (C)@ 95 (%) - TBP': 'TBP_TempAt_95pct',
    'Distillation Mass @ X Pct (C)@ 99 (%) - TBP': 'TBP_TempAt_99pct',

    'AromByWt (%)': 'AromWt_pct',
    'NaphthenesByWt (%)': 'NaphWt_pct',
    'ParaffinsByWt (%)': 'ParaWt_pct',

    'KinematicViscosity (cSt)@ 37.78 (C)': 'KV_37.78c_cSt',
    'KinematicViscosity (cSt)@ 98.89 (C)': 'KV_98.89c_cSt',
    'KinematicViscosity (cSt)@ 20 (C)': 'KV_20c_cSt',
    'KinematicViscosity (cSt)@ 40 (C)': 'KV_40c_cSt',
    'KinematicViscosity (cSt)@ 50 (C)': 'KV_50c_cSt',
    'KinematicViscosity (cSt)@ 100 (C)': 'KV_100c_cSt',
    'KinematicViscosity (cSt)@ 150 (C)': 'KV_150c_cSt'
}, inplace=True)
df.head()


,Crude_Name,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,...,AromWt_pct,NaphWt_pct,ParaWt_pct,KV_37.78c_cSt,KV_98.89c_cSt,KV_20c_cSt,KV_40c_cSt,KV_50c_cSt,KV_100c_cSt,KV_150c_cSt
0,Akpo-2014,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,...,24.571141,41.011754,34.417105,1.7573028543796,0.833438,2.091216,1.700584,1.443025,0.824673,NaN
1,AlJurf-2014,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,...,42.157942,36.772998,21.069060,7.05914911333451,2.025492,11.190540,6.703264,5.225721,1.991830,NaN
2,AlJurf-2015,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,...,39.122494,39.730427,21.147079,6.77137102006056,2.010330,10.544402,6.437194,5.049799,1.977528,NaN
3,Ashtart-2015,870.860260,1.002298,5.194113,0.233022,17.461417,84.608073,121.721566,248.007854,358.275744,...,40.484153,44.017708,15.498139,6.6515298282988,1.990580,10.208774,6.317605,4.927973,1.959541,NaN
4,Azeri-Light-Supsa-2019,846.458826,0.147956,0.735866,0.113251,4.265112,81.831627,120.955400,229.274959,320.215604,...,21.364167,58.922758,19.713075,3.93702798465944,1.841686,5.117420,3.810679,3.255558,1.818677,NaN


#### Duplicate Values, Missing Values & Data Types

In [224]:
#Checks for duplicate values
df.duplicated().sum()

np.int64(0)

In [225]:
#Checks for missing values
df.isna().sum()

Crude_Name                 0
StdLiqDensity_kgm3         0
SulfurWt_pct               0
ConradsonCarbon_wt_pct     0
NitrogenWt_pct             0
TBP_TempAt_1pct            0
TBP_TempAt_5pct            0
TBP_TempAt_10pct           0
TBP_TempAt_30pct           0
TBP_TempAt_50pct           0
TBP_TempAt_70pct           0
TBP_TempAt_90pct           0
TBP_TempAt_95pct           0
TBP_TempAt_99pct           0
AromWt_pct                 0
NaphWt_pct                 0
ParaWt_pct                 0
KV_37.78c_cSt              0
KV_98.89c_cSt              0
KV_20c_cSt                22
KV_40c_cSt                16
KV_50c_cSt                11
KV_100c_cSt               15
KV_150c_cSt               47
dtype: int64

In [226]:
#Deleting columns with more than 50% missing values
df.drop(columns='KV_150c_cSt', inplace=True)
df.shape

(53, 23)

In [227]:
#Exploring data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Crude_Name              53 non-null     object 
 1   StdLiqDensity_kgm3      53 non-null     float64
 2   SulfurWt_pct            53 non-null     float64
 3   ConradsonCarbon_wt_pct  53 non-null     float64
 4   NitrogenWt_pct          53 non-null     float64
 5   TBP_TempAt_1pct         53 non-null     float64
 6   TBP_TempAt_5pct         53 non-null     float64
 7   TBP_TempAt_10pct        53 non-null     float64
 8   TBP_TempAt_30pct        53 non-null     float64
 9   TBP_TempAt_50pct        53 non-null     float64
 10  TBP_TempAt_70pct        53 non-null     float64
 11  TBP_TempAt_90pct        53 non-null     float64
 12  TBP_TempAt_95pct        53 non-null     float64
 13  TBP_TempAt_99pct        53 non-null     float64
 14  AromWt_pct              53 non-null     floa

In [228]:
#Changing data types
df.loc[df['Crude_Name'] == 'Cinta-1983', 'KV_37.78c_cSt'] = 14.183399
df['KV_37.78c_cSt'] = pd.to_numeric(df['KV_37.78c_cSt'])

In [229]:
df = df[df['Crude_Name'] != 'Pisticci-2001']

#### ML Model Training

In [230]:
#Independent Features
X = df.drop(columns=['Crude_Name', 'AromWt_pct', 'NaphWt_pct', 'ParaWt_pct', 'KV_37.78c_cSt', 'KV_98.89c_cSt', 
                    'KV_20c_cSt', 'KV_40c_cSt', 'KV_50c_cSt', 'KV_100c_cSt'])
X.head()

,StdLiqDensity_kgm3,SulfurWt_pct,ConradsonCarbon_wt_pct,NitrogenWt_pct,TBP_TempAt_1pct,TBP_TempAt_5pct,TBP_TempAt_10pct,TBP_TempAt_30pct,TBP_TempAt_50pct,TBP_TempAt_70pct,TBP_TempAt_90pct,TBP_TempAt_95pct,TBP_TempAt_99pct
0,794.689603,0.070591,0.732083,0.062825,-16.244551,24.679712,74.031996,147.692242,231.447841,311.635864,465.560748,539.770951,687.654783
1,874.431106,1.826957,5.241924,0.168953,26.236785,98.856864,138.763063,251.728374,354.117493,468.885623,655.500149,748.288326,914.614065
2,872.972557,1.840926,5.365742,0.146255,26.408162,96.128866,135.041586,248.512058,351.412080,481.574377,655.699458,736.138972,885.381839
3,870.860260,1.002298,5.194113,0.233022,17.461417,84.608073,121.721566,248.007854,358.275744,468.225815,781.519485,949.416468,1161.723971
4,846.458826,0.147956,0.735866,0.113251,4.265112,81.831627,120.955400,229.274959,320.215604,423.105585,575.274504,642.156430,753.098296


In [231]:
#Dependent Features
y = df[['KV_37.78c_cSt', 'KV_98.89c_cSt']]

In [232]:
#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((41, 13), (11, 13), (41, 2), (11, 2))

In [233]:
#Define categorical and numerical feature columns
num_features = [col for col in X.columns]

#Define transformations for each feature
numeric_transformer = StandardScaler()                    

#Apply scaling to numeric features
preprocessor = ColumnTransformer([("StandardScaler", numeric_transformer, num_features)])


In [234]:
#Machine Learning Algorithms
models = {
    "Linear Regression": Pipeline([
        ('pre', preprocessor),
        
        ('linreg', MultiOutputRegressor(LinearRegression()))
    ]),
    "Lasso Regression": Pipeline([
        ('pre', preprocessor),
        ('lasso', MultiOutputRegressor(Lasso(alpha=1.0)))
    ]),
    "Ridge Regression": Pipeline([
        ('pre', preprocessor),
        ('ridge', MultiOutputRegressor(Ridge(alpha=1.0)))
    ]),
    "ElasticNet": Pipeline([
        ('pre', preprocessor),
        
        ('elasticnet', MultiOutputRegressor(ElasticNet(alpha=1.0)))
    ]),
    "SVR": Pipeline([
        ('pre', preprocessor),
        
        ('svr', MultiOutputRegressor(SVR(kernel='rbf', C=100, epsilon=0.1)))
    ]),
    "KNN": Pipeline([
        ('pre', preprocessor), 
        ('knn', MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3)))
    ]),
    "Decision Tree Regressor": Pipeline([
        ('pre', preprocessor),  
        ('dt', MultiOutputRegressor(DecisionTreeRegressor(random_state=42)))
    ]),
    "Extra Trees Regressor": Pipeline([
        ('pre', preprocessor),  
        ('et', MultiOutputRegressor(ExtraTreesRegressor(n_estimators=100, random_state=42)))
    ]),
    "Random Forest": Pipeline([
        ('pre', preprocessor),
        ('rf', MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42)))
    ]),
    "Gradient Boosting": Pipeline([
        ('pre', preprocessor),
        ('gb', MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)))
    ]),
    "Adaboost": Pipeline([
        ('pre', preprocessor),
        ('ab', MultiOutputRegressor(AdaBoostRegressor(n_estimators=100, random_state=42)))
    ]),
    "XGBoost": Pipeline([
        ('pre', preprocessor),
        ('xgb', MultiOutputRegressor(XGBRegressor(n_estimators=50, max_depth=3, random_state=42)))
    ]),
    
}

In [235]:
#Train and evaluate all models
print("Training ML Models...")

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'R2': r2_score(y_test, y_pred)
    }
results_df = pd.DataFrame(results).T
print(results_df.sort_values('R2', ascending=False))


Training ML Models...
                              MAE       RMSE        R2
Adaboost                 1.760472   2.744775  0.941052
Extra Trees Regressor    1.561591   3.721389  0.900933
Random Forest            2.117032   4.109767  0.860667
KNN                      1.955732   3.778545  0.806377
XGBoost                  2.131040   4.386559  0.805456
SVR                      2.244528   4.357607  0.735250
Decision Tree Regressor  2.154920   3.666496  0.605684
ElasticNet               5.856518   8.620361  0.332037
Gradient Boosting        3.584956  11.173313  0.175488
Ridge Regression         5.727874  10.343739  0.152360
Lasso Regression         5.800602   9.908712  0.139961
Linear Regression        6.577311  12.970495 -0.284308
